In [1]:
import pandas

In [2]:
df = pandas.read_csv('autos.csv', sep=",", encoding = 'Latin1')


df = df.drop(['seller', 'abtest', 'offerType', 'nrOfPictures', 'lastSeen', 'abtest', 'name'], axis=1)
print(df)

                dateCrawled  price vehicleType  yearOfRegistration    gearbox  \
0       2016-03-24 11:52:17    480         NaN                1993    manuell   
1       2016-03-24 10:58:45  18300       coupe                2011    manuell   
2       2016-03-14 12:52:21   9800         suv                2004  automatik   
3       2016-03-17 16:54:04   1500  kleinwagen                2001    manuell   
4       2016-03-31 17:25:20   3600  kleinwagen                2008    manuell   
...                     ...    ...         ...                 ...        ...   
371523  2016-03-14 17:48:27   2200         NaN                2005        NaN   
371524  2016-03-05 19:56:21   1199      cabrio                2000  automatik   
371525  2016-03-19 18:57:12   9200         bus                1996    manuell   
371526  2016-03-20 19:41:08   3400       kombi                2002    manuell   
371527  2016-03-07 19:39:19  28990   limousine                2013    manuell   

        powerPS        mode

In [3]:
#print(cats)
df['gearbox'].loc[df['gearbox'] == 'manuell'] = 1
df['gearbox'].loc[df['gearbox'] == 'automatik'] = 0
df = df.dropna(axis=0, subset=['gearbox'])

df['notRepairedDamage'].loc[df['notRepairedDamage'] == 'ja'] = 1
df['notRepairedDamage'].loc[df['notRepairedDamage'] == 'nein'] = 0
df = df.dropna(axis=0, subset=['notRepairedDamage'])

cats = pandas.get_dummies(df[['brand','vehicleType', 'fuelType', 'model']])
years_old = 2016 - df['yearOfRegistration']


df = df.drop(['yearOfRegistration'], axis = 1)
df = pandas.concat([df, cats, years_old], axis = 1)
df = df.drop(['brand','vehicleType', 'fuelType', 'model'], axis = 1)
df.head()




C:\Users\ELiu\AppData\Local\Continuum\miniconda3\envs\machine_learn\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,dateCrawled,price,gearbox,powerPS,kilometer,monthOfRegistration,notRepairedDamage,dateCreated,postalCode,brand_alfa_romeo,...,model_x_reihe,model_x_trail,model_x_type,model_xc_reihe,model_yaris,model_yeti,model_ypsilon,model_z_reihe,model_zafira,yearOfRegistration
1,2016-03-24 10:58:45,18300,1,190,125000,5,1,2016-03-24 00:00:00,66954,0,...,0,0,0,0,0,0,0,0,0,5
3,2016-03-17 16:54:04,1500,1,75,150000,6,0,2016-03-17 00:00:00,91074,0,...,0,0,0,0,0,0,0,0,0,15
4,2016-03-31 17:25:20,3600,1,69,90000,7,0,2016-03-31 00:00:00,60437,0,...,0,0,0,0,0,0,0,0,0,8
5,2016-04-04 17:36:23,650,1,102,150000,10,1,2016-04-04 00:00:00,33775,0,...,0,0,0,0,0,0,0,0,0,21
6,2016-04-01 20:48:51,2200,1,109,150000,8,0,2016-04-01 00:00:00,67112,0,...,0,0,0,0,0,0,0,0,0,12


In [4]:
# new changes

df = df.drop(['postalCode'], axis=1)

#create feature for days posted
# df[['dateCrawled','dateCreated']] = df[['dateCrawled','dateCreated']].apply(pandas.to_datetime)
# df['postedDays'] = (df['dateCrawled'] - df['dateCreated']).dt.days
# df['postedDays'].describe()

df = df.drop(['dateCrawled', 'dateCreated'], axis=1)

In [5]:
df.to_csv('processed.csv', index='False')